In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.3` // Or use any other 2.x version here
import $ivy.`sh.almond::almond-spark:0.6.0`

import org.apache.spark._
import org.apache.spark.sql.{functions => func, _}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

import org.slf4j.LoggerFactory
import org.apache.log4j.{Level, Logger}

val spark = NotebookSparkSession
      .builder()
      .config("spark.sql.join.preferSortMergeJoin", false)
      .config("spark.sql.shuffle.partitions", 64)
      .master("local[*]")
      .getOrCreate()

import spark.implicits._

Logger.getRootLogger().setLevel(Level.ERROR)

def run[A](code: => A): A = {
    val start = System.currentTimeMillis()
    val res = code
    println(s"Took ${System.currentTimeMillis() - start}")
    res
}

Loading spark-stubs
Getting spark JARs


log4j:WARN No appenders could be found for logger (org.eclipse.jetty.util.log).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.


Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
21/11/26 15:23:22 INFO SparkContext: Running Spark version 2.4.3
21/11/26 15:23:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/11/26 15:23:22 INFO SparkContext: Submitted application: 35ff8538-4932-4604-947a-10e91628f816
21/11/26 15:23:22 INFO SecurityManager: Changing view acls to: oscar
21/11/26 15:23:22 INFO SecurityManager: Changing modify acls to: oscar
21/11/26 15:23:22 INFO SecurityManager: Changing view acls groups to: 
21/11/26 15:23:22 INFO SecurityManager: Changing modify acls groups to: 
21/11/26 15:23:22 INFO SecurityManager: SecurityManager: authentication disabled; ui acls disabled; users  with view permissions: Set(oscar); groups with view permissions: Set(); users  with modify permissions: Set(oscar); groups with modify permissions: Set()
21/11/26 15:23:23 INFO Utils: Successfully started service 'sparkDrive

21/11/26 15:23:23 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/com/lihaoyi/mainargs_2.12/0.1.4/mainargs_2.12-0.1.4-sources.jar at spark://netrunner:36501/jars/mainargs_2.12-0.1.4-sources.jar with timestamp 1637936603502
21/11/26 15:23:23 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/com/lihaoyi/mainargs_2.12/0.1.4/mainargs_2.12-0.1.4.jar at spark://netrunner:36501/jars/mainargs_2.12-0.1.4.jar with timestamp 1637936603503
21/11/26 15:23:23 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/com/lihaoyi/os-lib_2.12/0.7.1/os-lib_2.12-0.7.1-sources.jar at spark://netrunner:36501/jars/os-lib_2.12-0.7.1-sources.jar with timestamp 1637936603503
21/11/26 15:23:23 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/com/lihaoyi/os-lib_2.12/0.7.1/os-lib_2.12-0.7.1.jar at spark://netrunner:36501/jars/os-lib_2.12-0.7.1.j

21/11/26 15:23:23 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/com/github/alexarchambault/argonaut-shapeless_6.2_2.12/1.2.0-M11/argonaut-shapeless_6.2_2.12-1.2.0-M11.jar at spark://netrunner:36501/jars/argonaut-shapeless_6.2_2.12-1.2.0-M11.jar with timestamp 1637936603508
21/11/26 15:23:23 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/org/eclipse/jetty/jetty-server/9.4.19.v20190610/jetty-server-9.4.19.v20190610.jar at spark://netrunner:36501/jars/jetty-server-9.4.19.v20190610.jar with timestamp 1637936603509
21/11/26 15:23:23 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/io/argonaut/argonaut_2.12/6.2.3/argonaut_2.12-6.2.3.jar at spark://netrunner:36501/jars/argonaut_2.12-6.2.3.jar with timestamp 1637936603510
21/11/26 15:23:23 INFO SparkContext: Added JAR file:/home/oscar/.cache/coursier/v1/https/repo1.maven.org/maven2/com/chuusai/shapeless_2.

Spark UI

import $ivy.$                                   // Or use any other 2.x version here

import $ivy.$                              


import org.apache.spark._

import org.apache.spark.sql.{functions => func, _}

import org.apache.spark.sql.functions._

import org.apache.spark.sql.types._


import org.slf4j.LoggerFactory

import org.apache.log4j.{Level, Logger}


spark: SparkSession = org.apache.spark.sql.SparkSession@b82d054
import spark.implicits._


defined function run

In [2]:
import org.apache.spark.sql.expressions.Window

import org.apache.spark.sql.expressions.Window

In [61]:
val races = spark.read.format("csv")
    .option("header", "true")
    .option("sep", ",")
    .load("../data/races.csv")
    .where(col("year") === 1996)


load at cmd60.sc:4

1 / 1

races: Dataset[Row] = [raceId: string, year: string ... 6 more fields]

In [62]:
val lapTimeToMs = (time: String) => {
    val regex = """([0-9]|[0-9][0-9]):([0-9][0-9])\.([0-9][0-9][0-9])""".r
    time match {
        case regex(min,sec,ms) => min.toInt * 60 * 1000 + sec.toInt * 1000 + ms.toInt
        case "\\N" => 180000
    }
    
}: Long

lapTimeToMs: String => Long = ammonite.$sess.cmd61$Helper$$Lambda$6481/1274941098@24c9ce3c

In [63]:
val lapTimeToMsUDF = udf(lapTimeToMs)
spark.udf.register("lapTimeToMs", lapTimeToMsUDF)

lapTimeToMsUDF: expressions.UserDefinedFunction = UserDefinedFunction(
  ammonite.$sess.cmd61$Helper$$Lambda$6481/1274941098@24c9ce3c,
  LongType,
  Some(List(StringType))
)
res62_1: expressions.UserDefinedFunction = UserDefinedFunction(
  ammonite.$sess.cmd61$Helper$$Lambda$6481/1274941098@24c9ce3c,
  LongType,
  Some(List(StringType))
)

In [64]:
val msToLapTime = (time: Long) => {
    val mins = time / 60000
    val secs = (time - mins*60000)/1000
    val ms = time - mins*60000 - secs*1000
    
    val formattedSecs = if((secs / 10).toInt == 0) "0" + secs else secs
    // if ms = 00x -> "0"+"0"+x . if ms = 0xx -> "0"+ms
    val formattedMs = if((ms / 100).toInt == 0) "0" + (if((ms / 10).toInt == 0) "0" + ms else ms) else ms
    mins + ":" + formattedSecs + "." + formattedMs    
}: String

msToLapTime: Long => String = ammonite.$sess.cmd63$Helper$$Lambda$6489/1800869518@2172ffe2

In [65]:
val msToLapTimeUDF = udf(msToLapTime)
spark.udf.register("msToLapTime", msToLapTimeUDF)

msToLapTimeUDF: expressions.UserDefinedFunction = UserDefinedFunction(
  ammonite.$sess.cmd63$Helper$$Lambda$6489/1800869518@2172ffe2,
  StringType,
  Some(List(LongType))
)
res64_1: expressions.UserDefinedFunction = UserDefinedFunction(
  ammonite.$sess.cmd63$Helper$$Lambda$6489/1800869518@2172ffe2,
  StringType,
  Some(List(LongType))
)

In [66]:
val driverWindow = Window.partitionBy("driverId")

val avg_lap_times = spark.read.format("csv")
    .option("header", "true")
    .option("sep", ",")
    .load("../data/lap_times.csv")
    .withColumnRenamed("time", "lapTime")
    .join(races, Seq("raceId"), "right")
    .withColumn("milliseconds", col("milliseconds").cast(IntegerType))
    .withColumn("avgMs", avg(col("milliseconds")).over(driverWindow))
    .dropDuplicates("driverId")
    .select("driverId", "avgMs")


load at cmd65.sc:6

1 / 1

driverWindow: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@5d188929
avg_lap_times: DataFrame = [driverId: string, avgMs: double]

In [67]:
val drivers = spark.read.format("csv")
    .option("header", "true")
    .option("sep", ",")
    .load("../data/drivers.csv")

load at cmd66.sc:4

1 / 1

drivers: DataFrame = [driverId: string, driverRef: string ... 7 more fields]

In [68]:
val seasonWindow = Window.partitionBy("year")

val lapCount = spark.read.format("csv")
    .option("header", "true")
    .option("sep", ",")
    .load("../data/lap_times.csv")
    .join(races, Seq("raceId"), "right")
    .withColumn("lapsPerDriver", count(col("lap")).over(driverWindow))

val (distinctDrivers, allLaps) = lapCount
    .agg(
        countDistinct("driverID"),
        count(col("lap"))
    ).as[(BigInt, BigInt)]
    .collect()(0)

val avgLapsThisPeriod = allLaps.toInt / distinctDrivers.toInt


//filtrar con isInCollection vv es igual a hacer un join left-anti
val experiencedDrivers = lapCount
    .where(col("lapsPerDriver") >= avgLapsThisPeriod)
    .select("driverId")
    .distinct()
    .as[String]
    .collect()

load at cmd67.sc:6

1 / 1

run at ThreadPoolExecutor.java:1149

4 / 4

collect at cmd67.sc:14

...

collect at cmd67.sc:25

1 / 1

collect at cmd67.sc:25

64 / 64

seasonWindow: expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@76cef5d1
lapCount: DataFrame = [raceId: string, driverId: string ... 12 more fields]
distinctDrivers: BigInt = 24
allLaps: BigInt = 14431
avgLapsThisPeriod: Int = 601
experiencedDrivers: Array[String] = Array(
  "64",
  "30",
  "83",
  "49",
  "84",
  "57",
  "55",
  "44",
  "79",
  "73",
  "35",
  "77",
  "71",
  "14",
  "63",
  "65",
  "22"
)

In [69]:
val mostConsistentDriver = spark.read.format("csv")
    .option("header", "true")
    .option("sep", ",")
    .load("../data/results.csv")
    .join(races, Seq("raceId"), "right")
    .na.drop(Seq("fastestLapTime"))
//     .withColumn("fastestLapTime", regexp_replace(col("fastestLapTime"), "[0-9]|[0-9][0-9]:[0-9][0-9]\\.[0-9][0-9][0-9]", "."))
//     .distinct().show()
    .withColumn("fastestLapTimeMs", lapTimeToMsUDF(col("fastestLapTime")))
    .withColumn("avgFastestLapMs", avg(col("fastestLapTimeMs")).over(driverWindow))
    .dropDuplicates("driverId")
    .join(avg_lap_times, Seq("driverId"), "left")
    .withColumn("diffLapTimes", abs('avgMs - 'avgFastestLapMs).cast(IntegerType))
    .withColumn("avgDiff", msToLapTimeUDF(col("diffLapTimes").cast(IntegerType)))
    .where(col("driverId").isInCollection(experiencedDrivers))
    .join(drivers, "driverId")
    .withColumn("driver", concat(col("forename"), lit(" "), col("surname")))
    .select("driver", "avgDiff")
    .orderBy("avgDiff")


load at cmd68.sc:4

1 / 1

mostConsistentDriver: Dataset[Row] = [driver: string, avgDiff: string]

In [70]:
mostConsistentDriver.show(300)

run at ThreadPoolExecutor.java:1149

1 / 1

run at ThreadPoolExecutor.java:1149

4 / 4

run at ThreadPoolExecutor.java:1149

...

show at cmd69.sc:1

1 / 1

show at cmd69.sc:1

64 / 64

+--------------------+--------+
|              driver| avgDiff|
+--------------------+--------+
|          Pedro Lamy|1:21.933|
|           Mika Salo|1:22.551|
|         Pedro Diniz|1:23.104|
|      Ricardo Rosset|1:23.119|
|Heinz-Harald Fren...|1:23.816|
|       Ukyo Katayama|1:23.897|
|  Michael Schumacher|1:24.050|
|      Martin Brundle|1:24.555|
|  Rubens Barrichello|1:24.700|
|      Gerhard Berger|1:25.007|
|     David Coulthard|1:25.440|
|       Mika Häkkinen|1:25.754|
|          Jean Alesi|1:25.899|
|       Olivier Panis|1:25.939|
|  Jacques Villeneuve|1:26.028|
|      Johnny Herbert|1:26.097|
|          Damon Hill|1:26.915|
+--------------------+--------+

